In [15]:
import cv2
import numpy as np
import math
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
import random
from mpl_toolkits.mplot3d import Axes3D

In [43]:
color_selection_path = r"\\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT"
images_to_process = r"\\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC"
processed_images_path = r"\\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static"
downscale_factor = 2 #only for tile creation
tile_size = 200
mask_output_type = ".png"
file_type = ".png"

In [17]:

# Global variables
drawing = False
ix, iy = -1, -1
mask = None  # Initialize mask
save_rectangle = False

zoom_factor = 1.0
pan_x, pan_y = 0, 0
cursor_x, cursor_y = 0, 0
mouseRGB_last_x = 0
mouseRGB_last_y = 0
coordinates = []

def mouseRGB(event, x, y, flags, param):
    global zoom_factor, pan_x, pan_y, cursor_x, cursor_y, drawing, mouseRGB_last_x, mouseRGB_last_y, display_img, img, downscale_factor, coordinates

    if event == cv2.EVENT_LBUTTONDOWN:
        mouseRGB_last_x, mouseRGB_last_y = x, y

    elif event == cv2.EVENT_RBUTTONDOWN:
        edge_len = tile_size * downscale_factor
        click_center_x, click_center_y = x, y

        M = np.float32([[zoom_factor, 0, cursor_x - cursor_x * zoom_factor + pan_x], [0, zoom_factor, cursor_y - cursor_y * zoom_factor + pan_y]])
        inv_M = cv2.invertAffineTransform(M)
        orig_center = cv2.transform(np.array([[[click_center_x, click_center_y]]]), inv_M)[0][0]

        orig_top_left = (int(orig_center[0] - edge_len // 2), int(orig_center[1] - edge_len // 2))
        orig_bottom_right = (int(orig_center[0] + edge_len // 2), int(orig_center[1] + edge_len // 2))

        cv2.rectangle(img,
                      orig_top_left,
                      orig_bottom_right,
                      (0, 0, 255), 2)
        coordinates.append([orig_top_left[0], orig_top_left[1], orig_bottom_right[0], orig_bottom_right[1]])
        update_display_img()

    elif event == cv2.EVENT_MOUSEWHEEL:
        cursor_x, cursor_y = x, y
        zoom_factor += 0.1 if flags > 0 else -0.1
        zoom_factor = max(0.1, zoom_factor)
        update_display_img()

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            pan_x += x - mouseRGB_last_x
            pan_y += y - mouseRGB_last_y
            mouseRGB_last_x, mouseRGB_last_y = x, y
            update_display_img()

def update_display_img():
    global display_img, zoom_factor, pan_x, pan_y, cursor_x, cursor_y, img
    h, w = img.shape[:2]
    center_x, center_y = cursor_x, cursor_y
    M = np.float32([[zoom_factor, 0, center_x - center_x * zoom_factor + pan_x],
                    [0, zoom_factor, center_y - center_y * zoom_factor + pan_y]])
    display_img = cv2.warpAffine(img, M, (w, h))
    cv2.imshow('image', display_img)

# Load images from the directory
mypath = color_selection_path
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
# Randomly select 9 files
if len(onlyfiles) > 4:
    selected_files = random.sample(onlyfiles, 4)
else:
    selected_files = onlyfiles
images = []

for read_path in selected_files:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img_full = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img_full is None:
        print(f"Failed to load image at {final_path}")
        continue

    # Initialize/reset global variables
    drawing = False
    ix, iy = -1, -1
    mask = None  # Initialize mask
    save_rectangle = False

    zoom_factor = 1.0
    pan_x, pan_y = 0, 0
    cursor_x, cursor_y = 0, 0
    mouseRGB_last_x = 0
    mouseRGB_last_y = 0

    height, width = img_full.shape[:2]
    print(height, width)
    img = cv2.resize(img_full, (int(width * downscale_factor), int(height * downscale_factor)), interpolation=cv2.INTER_LINEAR)
    display_img = None
    coordinates = []

    cv2.imshow('image', img)
    cv2.setMouseCallback('image', mouseRGB)

    # Wait for ESC key to exit or 'x' to save rectangles
    while True:
        key = cv2.waitKey(20) & 0xFF
        if key == 27:  # ESC key
            break
        elif key == ord('x'):
            save_rectangle = True  # Change color to blue

    # Process rectangles and save cropped images
    for coord in coordinates:
        print(f"Cropping coordinates: {coord}")
        top_left = (int(coord[0] / downscale_factor), int(coord[1] / downscale_factor))
        bottom_right = (int(coord[2] / downscale_factor), int(coord[3] / downscale_factor))
        cropped_image = img_full[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        if cropped_image.size == 0:
            print(f"Warning: Cropped image is empty for coordinates: {coord}")
            continue
        print(cropped_image.shape)
        images.append(cropped_image)

    cv2.destroyAllWindows()


Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\152_tile_00011.png
256 256
Cropping coordinates: [39, 49, 439, 449]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\214_tile_00002.png
256 256
Cropping coordinates: [46, 49, 446, 449]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\142_tile_00022.png
256 256
Cropping coordinates: [67, 51, 467, 451]
(200, 200, 3)
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_Ben2\IHC_GT\51__tile_00076.png
256 256
Cropping coordinates: [43, 63, 443, 463]
(200, 200, 3)


In [18]:
def concatenate_images(images, rows, cols):
    """
    Concatenate multiple images of the same size into a grid.
    :param images: List of images to concatenate
    :param rows: Number of rows in the grid
    :param cols: Number of columns in the grid
    :return: Concatenated image
    """
    assert len(images) <= rows * cols, "Number of images is greater than grid size"

    # Ensure all images are the same size
    assert all(img.shape == images[0].shape for img in images), "All images must have the same dimensions"

    # Create rows of images
    rows_of_images = [cv2.hconcat(images[i:i+cols]) for i in range(0, len(images), cols)]

    # Concatenate rows vertically
    return cv2.vconcat(rows_of_images)

num_imgs = len(images)
x_grid = int(math.sqrt(num_imgs))
if num_imgs % x_grid == 0:
    y_grid = x_grid
else:
    y_grid = x_grid + 1

if images:
    result = concatenate_images(images, y_grid, x_grid)
else:
    print("No images to concatenate.")

In [19]:
# Initialize lists and index
stain_colors = ["brown", "blue", "red"]
hsv_values = [[] for _ in stain_colors]  # List to store HSV values for each color
hsv_index = 0

def set_hsv_index(value):
    global hsv_index
    try:
        hsv_index = int(value)
        if 0 <= hsv_index < len(stain_colors):
            print(f"HSV index set to: {hsv_index} ({stain_colors[hsv_index]})")
        else:
            print(f"Invalid index. Please enter a number between 0 and {len(stain_colors) - 1}.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

# Function to display the HSV values on mouse click
def get_hsv_values(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN or event == cv2.EVENT_RBUTTONDOWN:
        hsv_value = hsv_image[y, x]
        print(f"Mouse Event: {event}, HSV Value at ({x}, {y}): {hsv_value}")
        if event == cv2.EVENT_RBUTTONDOWN:
            if 0 <= hsv_index < len(stain_colors):
                hsv_values[hsv_index].append(hsv_value)
                print(f"HSV Value added to index {hsv_index}: {hsv_value}")
            else:
                print("Invalid HSV index. Cannot add value.")

# Convert the image to HSV color space
hsv_image = cv2.cvtColor(result, cv2.COLOR_BGR2HSV)

# Create a resizable window and set a mouse callback
cv2.namedWindow('Image', cv2.WINDOW_KEEPRATIO)
cv2.setMouseCallback('Image', get_hsv_values)
input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
set_hsv_index(input_value)

while True:
    cv2.imshow('Image', result)
    key = cv2.waitKey(20) & 0xFF
    if key == 27:  # ESC key to break
        break
    elif key == ord('c'):  # Press 'c' key to change HSV index
        # Ask the user to input the HSV index
        input_value = input("Enter a number for HSV index (0 for brown, 1 for blue, 2 for red): ")
        set_hsv_index(input_value)

cv2.destroyAllWindows()

# Initialize lists to store max and min values for each color category
max_values_list = []
min_values_list = []

# Calculate max and min values separately for each color category
for i in hsv_values:  # Iterating over each color category
    if i:  # Ensure there are values to compute max and min
        max_values = np.amax(i, axis=0).tolist()  # Convert to list for better readability
        min_values = np.amin(i, axis=0).tolist()  # Convert to list for better readability
        max_values_list.append(max_values)
        min_values_list.append(min_values)
    else:
        max_values_list.append([0, 0, 0])  # Default values if no data
        min_values_list.append([0, 0, 0])  # Default values if no data

print("Max values for each color category:", max_values_list)
print("Min values for each color category:", min_values_list)

HSV index set to: 0 (brown)
Mouse Event: 1, HSV Value at (130, 149): [ 13 131 179]
Mouse Event: 2, HSV Value at (130, 149): [ 13 131 179]
HSV Value added to index 0: [ 13 131 179]
Mouse Event: 2, HSV Value at (130, 151): [ 13  76 202]
HSV Value added to index 0: [ 13  76 202]
Mouse Event: 2, HSV Value at (131, 150): [ 12 130 129]
HSV Value added to index 0: [ 12 130 129]
Mouse Event: 1, HSV Value at (110, 17): [  8  74 183]
Mouse Event: 2, HSV Value at (110, 17): [  8  74 183]
HSV Value added to index 0: [  8  74 183]
Mouse Event: 1, HSV Value at (112, 14): [ 10 100 156]
Mouse Event: 2, HSV Value at (112, 14): [ 10 100 156]
HSV Value added to index 0: [ 10 100 156]
Mouse Event: 1, HSV Value at (153, 394): [  3 151  59]
Mouse Event: 2, HSV Value at (153, 394): [  3 151  59]
HSV Value added to index 0: [  3 151  59]
Mouse Event: 1, HSV Value at (170, 393): [ 11 153 120]
Mouse Event: 2, HSV Value at (170, 393): [ 11 153 120]
HSV Value added to index 0: [ 11 153 120]
Mouse Event: 1, HSV Va

In [20]:
import plotly.graph_objects as go
from plotly.offline import plot

# Convert the image from BGR to RGB format (OpenCV loads images in BGR format)
plot_img = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# Get image dimensions
height, width, _ = plot_img.shape

# Generate points every 10 pixels
step_size = 10
x_coords = np.arange(0, width, step_size)
y_coords = np.arange(0, height, step_size)
grid_x, grid_y = np.meshgrid(x_coords, y_coords)
sampled_points = np.column_stack((grid_x.ravel(), grid_y.ravel()))

# Sample the colors at the points
colors = plot_img[sampled_points[:, 1], sampled_points[:, 0]]
print(colors)
fig = go.Figure()
# Add the regular-sized points
fig.add_trace(go.Scatter3d(
    x=colors[:, 0],
    y=colors[:, 1],
    z=colors[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=['rgb({}, {}, {})'.format(r, g, b) for r, g, b in colors]
    ),
    name='Regular Points'
))
hsv_brown = hsv_values[0]
hsv_blue = hsv_values[1]
hsv_red = hsv_values[2]

def hsv_to_rgb(hsv_list):
    # Convert the list to a numpy array
    hsv_array = np.array(hsv_list, dtype=np.uint8)

    # Reshape the array to have a single row with many pixels
    hsv_array = hsv_array.reshape((1, -1, 3))

    # Convert HSV to BGR
    bgr_array = cv2.cvtColor(hsv_array, cv2.COLOR_HSV2BGR)

    # Convert BGR to RGB
    rgb_array = cv2.cvtColor(bgr_array, cv2.COLOR_BGR2RGB)

    # Reshape back to a list of RGB values
    rgb_list = rgb_array.reshape(-1, 3).tolist()

    return rgb_list


# Convert HSV values to RGB
rgb_brown = hsv_to_rgb(hsv_brown)
rgb_blue = hsv_to_rgb(hsv_blue)
rgb_red = hsv_to_rgb(hsv_red)
all_rgb = np.vstack((rgb_brown, rgb_blue, rgb_red))
fig.add_trace(go.Scatter3d(
    x=all_rgb[:, 0],
    y=all_rgb[:, 1],
    z=all_rgb[:, 2],
    mode='markers',
    marker=dict(
        size=20,  # Increase size for larger points
        color=['rgb({}, {}, {})'.format(r, g, b) for r, g, b in all_rgb]
    ),
    name='Larger Points'
))

# Set plot labels
fig.update_layout(
    scene=dict(
        xaxis_title='Red',
        yaxis_title='Green',
        zaxis_title='Blue'
    ),
    title='3D Color Space'
)

# Save the plot as an HTML file and open it in the browser
plot(fig, filename='3d_color_space.html', auto_open=True)


[[237 236 239]
 [229 227 229]
 [244 243 245]
 ...
 [228 223 227]
 [243 240 243]
 [177 185 205]]


'3d_color_space.html'

In [21]:
import numpy as np
import cv2
import plotly.graph_objects as go
from plotly.offline import plot

def rgb_to_hsv(rgb_array):
    """
    Convert an array of RGB values to HSV values.

    Parameters:
    - rgb_array: A numpy array of RGB values with shape (N, 3), where N is the number of RGB values.

    Returns:
    - hsv_array: A numpy array of HSV values with the same shape as the input.
    """
    # Ensure the input is a numpy array
    rgb_array = np.array(rgb_array, dtype=np.uint8)

    # Convert RGB to BGR (OpenCV uses BGR by default)
    bgr_array = rgb_array[:, [2, 1, 0]]

    # Reshape the array to a 1xN image (for OpenCV compatibility)
    bgr_image = bgr_array.reshape((1, -1, 3))

    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)

    # Reshape back to the original shape
    hsv_array = hsv_image.reshape(-1, 3)

    return hsv_array

def hsv_to_rgb(hsv_array):
    """
    Convert an array of HSV values to RGB values.

    Parameters:
    - hsv_array: A numpy array of HSV values with shape (N, 3), where N is the number of HSV values.

    Returns:
    - rgb_array: A numpy array of RGB values with the same shape as the input.
    """
    # Ensure the input is a numpy array
    hsv_array = np.array(hsv_array, dtype=np.uint8)

    # Reshape the array to a 1xN image (for OpenCV compatibility)
    hsv_image = hsv_array.reshape((1, -1, 3))

    # Convert HSV to BGR
    bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    # Convert BGR to RGB
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

    # Reshape back to the original shape
    rgb_array = rgb_image.reshape(-1, 3)

    return rgb_array

# Assuming 'result' is your image array
plot_img = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# Get image dimensions
height, width, _ = plot_img.shape

# Generate points every 10 pixels
step_size = 10
x_coords = np.arange(0, width, step_size)
y_coords = np.arange(0, height, step_size)
grid_x, grid_y = np.meshgrid(x_coords, y_coords)
sampled_points = np.column_stack((grid_x.ravel(), grid_y.ravel()))

# Sample the colors at the points
colors = plot_img[sampled_points[:, 1], sampled_points[:, 0]]
hsv_sampled_colors = rgb_to_hsv(colors)
print("HSV Sampled Colors:\n", hsv_sampled_colors)

# Normalize HSV values to the range [0, 1] for Plotly
hsv_sampled_colors = hsv_sampled_colors.astype(float)

# Create the figure
fig = go.Figure()

# Add the regular-sized points
fig.add_trace(go.Scatter3d(
    x=hsv_sampled_colors[:, 0],
    y=hsv_sampled_colors[:, 1],
    z=hsv_sampled_colors[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=['rgb({}, {}, {})'.format(*colors[i]) for i in range(len(colors))]
    ),
    name='Regular Points'
))

# Assuming hsv_values is defined elsewhere
hsv_brown = hsv_values[0]
hsv_blue = hsv_values[1]
hsv_red = hsv_values[2]
all_HSV = np.asarray(hsv_brown + hsv_blue + hsv_red)

# Convert HSV values to RGB
all_RGB = hsv_to_rgb(all_HSV)
all_RGB = np.asarray(all_RGB)

# Normalize HSV values to the range [0, 1] for Plotly
all_HSV = all_HSV.astype(float)
all_HSV[:, 0]  # Hue
all_HSV[:, 1]  # Saturation
all_HSV[:, 2]  # Value

# Add the larger points
fig.add_trace(go.Scatter3d(
    x=all_HSV[:, 0],  # Normalize
    y=all_HSV[:, 1],  # Normalize
    z=all_HSV[:, 2],  # Normalize
    mode='markers',
    marker=dict(
        size=20,  # Increase size for larger points
        color=['rgb({}, {}, {})'.format(*all_RGB[i]) for i in range(len(all_RGB))]
    ),
    name='Larger Points'
))

# Set plot labels
fig.update_layout(
    scene=dict(
        xaxis_title='Hue',
        yaxis_title='Saturation',
        zaxis_title='Value'
    ),
    title='3D Color Space'
)

# Save the plot as an HTML file and open it in the browser
plot(fig, filename='3d_color_space.html', auto_open=True)


HSV Sampled Colors:
 [[130   3 239]
 [150   2 229]
 [135   2 245]
 ...
 [156   6 228]
 [150   3 243]
 [111  35 205]]


'3d_color_space.html'

In [44]:
def binary_mask_to_xy(binary_mask):
    # Find the coordinates of white pixels (value 1)
    white_points = np.argwhere(binary_mask == 255)

    # Create an array to hold the xy coordinates
    xy_array = np.zeros((white_points.shape[0], 2), dtype=int)

    # Fill the array with x, y coordinates
    xy_array[:, 0] = white_points[:, 1]  # x coordinate (column)
    xy_array[:, 1] = white_points[:, 0]  # y coordinate (row)

    return xy_array

def imfill(img):
    # Create a mask larger than the image
    h, w = img.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    # Floodfill from the corner
    filled_image = img.copy()
    cv2.floodFill(filled_image, mask, (0,0), 255)

    # Invert the filled image
    filled_image_inv = cv2.bitwise_not(filled_image)

    # Combine the filled image with the original image
    result = img | filled_image_inv

    return result

mypath = images_to_process
only_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for read_path in only_files:
    final_path = join(mypath, read_path)
    print(f"Processing {final_path}")  # Debugging line to print the file path being processed

    img = cv2.imread(final_path, cv2.IMREAD_COLOR)

    if img is None:
        print(f"Failed to load image at {final_path}")
        continue
    blurred_img = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blurred_img, cv2.COLOR_BGR2HSV)

    # Define the range of blue color in HSV
    low_blue = np.array([55, 50, 50])
    high_blue = np.array([130, 255, 255])
    #low_blue = np.asarray(min_values_list[2])
    #high_blue = np.asarray(max_values_list[2])


    # Define the range of red color in HSV
    low_red = np.array([130, 50, 50])
    high_red = np.array([180, 255, 255])
    #low_red = np.asarray(min_values_list[1])
    #high_red = np.asarray(max_values_list[1])

    # Define HSV range for brown color
    low_brown = np.array([0, 50, 40])
    high_brown = np.array([22, 100, 240])
    #low_brown = np.asarray(min_values_list[0])
    #high_brown = np.asarray(max_values_list[0])

    # Create masks based on the color ranges
    blue_mask = cv2.inRange(hsv, low_blue, high_blue)
    #filled_blue_mask = imfill(blue_mask)
    filled_blue_mask = blue_mask
    red_mask = cv2.inRange(hsv, low_red, high_red)
    #filled_red_mask = imfill(red_mask)
    filled_red_mask = red_mask
    brown_mask = cv2.inRange(hsv, low_brown, high_brown)
    #filled_brown_mask = imfill(brown_mask)
    filled_brown_mask = brown_mask

    # Create a blank image with the same dimensions as the original image
    overlay = np.zeros_like(img)

    # Assign colors to the overlay based on masks
    overlay[filled_blue_mask != 0] = [255, 0, 0]  # Blue mask as red (BGR format)
    overlay[filled_red_mask != 0] = [0, 0, 255]  # Red mask as blue (BGR format)
    overlay[filled_brown_mask != 0] = [0, 255, 0]  # Brown mask as custom color (BGR format)

    # Create the logical image
    logical_image = np.zeros_like(img[:, :, 0], dtype=np.uint8)
    logical_image[filled_blue_mask != 0] = 2  # Blue stain
    logical_image[filled_red_mask != 0] = 1  # Red stain
    logical_image[filled_brown_mask != 0] = 3  # Brown stain
    logical_image[(logical_image == 0) & (filled_blue_mask == 0) & (filled_red_mask == 0) & (filled_brown_mask == 0)] = 4  # Other

    # Combine the overlay with the original image
    masked_image = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    # Create paths for masks and logicals folders
    out_path_masks = os.path.join(processed_images_path, 'masks')
    out_path_logicals = os.path.join(processed_images_path, 'logicals')
    if not os.path.exists(out_path_masks):
        os.makedirs(out_path_masks)
    if not os.path.exists(out_path_logicals):
        os.makedirs(out_path_logicals)
    logical_path = join(out_path_logicals, read_path.replace(file_type, '_logical.tif'))
    cv2.imwrite(logical_path, logical_image)
    full_path_masks = join(out_path_masks, read_path.replace(file_type, mask_output_type))
    cv2.imwrite(full_path_masks, masked_image)
    print(f"Saved processed image to {full_path_masks}")  # Debugging line to confirm save"""

Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC\63__tile_00006_inferred.png
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static\masks\63__tile_00006_inferred.png
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC\026_tile_00074_inferred.png
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static\masks\026_tile_00074_inferred.png
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC\020_tile_00079_inferred.png
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static\masks\020_tile_00079_inferred.png
Processing \\10.99.68.178\ashleyex\Type_1_diabetes\HE2IHC\val_test_S_IHC\147_tile_00016_inferred.png
Saved processed image to \\10.17.97.73\kiemen-lab-data\Yu Shen\T1D\6521_aab\svs_files\ben_processed_Synthetic_static\masks\147_tile_00016_infer